Na koncu iz prenešenih html strani izluščimo še podatke o sestavinah in postopku priprave ter jih shranimo v 2 novi csv datoteki "sestavine.csv" in "podatki.csv".

In [52]:
import pandas as pd
import os
from tqdm import tqdm
import re
import csv

In [53]:
naslovi_receptov = pd.read_csv('podatki_z_indeksi.csv')

,Unnamed: 0,povezava,ime,avtor,cas_priprave,povprečna_ocena,kategorija
0,0,'/recepti/juhe-in-zakuhe/ostale-juhe/cmocki-sa...,Cmočki šarenke v žametni bučni juhi,Spar Slovenija,45 min,Unknown,juha
1,1,'/recepti/juhe-in-zakuhe/ostale-juhe/spargljev...,Špargljeva kremna juha z žličniki iz šunke in ...,HOFER Slovenija,45 min,Unknown,juha
2,2,'/recepti/juhe-in-zakuhe/ostale-juhe/skutn-ca/...,Skutn'ca,ZELENE DOLINE,15 min,Unknown,juha
3,3,'/recepti/juhe-in-zakuhe/jusne-zakuhe/domaca-r...,Domača ribana kaša,Žito,30 min,Unknown,juha
4,4,'/recepti/juhe-in-zakuhe/hladne-juhe/hladna-ku...,Hladna kumarična juha z morskim nabodalom,HOFER Slovenija,30 min,Unknown,juha
...,...,...,...,...,...,...,...
12761,12761,'/recepti/sladice/palacinke/palacinke-z-jabolc...,Palačinke z jabolčnim sirupom in ginom,percya,30 min,Unknown,sladice
12762,12762,'/recepti/sladice/palacinke/lesnikove-palacink...,Lešnikove palačinke,Kitchen bee,15 min,Unknown,sladice
12763,12763,'/recepti/sladice/pecivo/pecivo-5-4/22612/',Pecivo 5/4,Miss Pariz,1 ura,Unknown,sladice
12764,12764,'/recepti/sladice/pecivo/cokoladna-tortica-z-r...,Čokoladna tortica z rdečim vinom v skodelici,Neza V.,30 min,Unknown,sladice


Zagotovimo, da program najprej preveri, koliko strani smo že obdelali in shranili podatke o njihovih sestavinah oz. postopkih priprave, da se stvari ne ponavljajo, če program poženemo večkrat.

In [47]:
sestavine = pd.read_csv('sestavine.csv', header = None)

sestavine = sestavine.set_axis(['indeks', 'količina', 'sestavina'], axis=1, inplace=False)

prebrane_sestavine = set(sestavine['indeks'])

C:\Users\Marija\AppData\Local\Temp\ipykernel_7180\2444434152.py:3: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  sestavine = sestavine.set_axis(['indeks', 'količina', 'sestavina'], axis=1, inplace=False)


Nato podatke preberemo iz html-jev in shtsnimo v ustrezno csv datoteko.

In [49]:
with open('sestavine.csv', 'a', encoding='utf-8') as d:
    writer = csv.DictWriter(d, fieldnames=['indeks', 'količina', 'sestavina'])
    for vrstica in tqdm(naslovi_receptov.iloc, total=len(naslovi_receptov)):
        id = vrstica['Unnamed: 0']
        if int(id) in prebrane_sestavine:
            continue
        pot = f"html_strani/{id}.html"
        if not os.path.exists(pot):
            print(f"{id} ne obstaja.")
        with open(pot, 'r', encoding='utf-8') as d:
            vsebina = d.read()
        vzorec_sestavine = re.compile(r"""<p.*?itemprop="recipeIngredient"><span class="label">(?P<kolicina>.*?)</span><span class="label-value">(?P<sestavina>.*?)</span></p>""", re.DOTALL)
        sestavine = re.findall(vzorec_sestavine, vsebina)
        if len(sestavine) == 0:
            print(f"{id}: ni nasel sestavin")
            continue

        for sestavina in sestavine:
            sestavina_slovar = {'indeks': id, 'količina': sestavina[0], 'sestavina': sestavina[1]}
            writer.writerow(sestavina_slovar)

100%|██████████| 12766/12766 [48:47<00:00,  4.36it/s]  


In [56]:
postopki = pd.read_csv('postopki.csv', header = None)

postopki = postopki.set_axis(['indeks', 'korak', 'postopek'], axis=1, inplace=False)

prebrani_postopki = set(postopki['indeks'])

C:\Users\Marija\AppData\Local\Temp\ipykernel_7180\1458954422.py:3: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  postopki = postopki.set_axis(['indeks', 'korak', 'postopek'], axis=1, inplace=False)


In [58]:
with open('postopki.csv', 'a', encoding='utf-8') as d:
    writer = csv.DictWriter(d, fieldnames=['indeks', 'korak', 'postopek'])
    for vrstica in tqdm(naslovi_receptov.iloc, total=len(naslovi_receptov)):
            id = vrstica['Unnamed: 0']
            if int(id) in prebrani_postopki:
                continue
            pot = f"html_strani/{id}.html"
            if not os.path.exists(pot):
                print(f"{id} ne obstaja.")
            with open(pot, 'r', encoding='utf-8') as d:
                vsebina = d.read()
            vzorec_postopek = re.compile(r"""<p.*?itemprop="recipeInstructions"><span class="label">(?P<korak>.*?)</span><span class="+data"+>(?P<postopek>.*?)</span></p>""", re.DOTALL)
            navodila = re.findall(vzorec_postopek, vsebina)
            if len(navodila) == 0:
                print(f"{id}: ni nasel sestavin")
                continue

            for navodilo in navodila:
                navodilo_slovar = {'indeks': id, 'korak': navodilo[0], 'postopek': navodilo[1]}
                writer.writerow(navodilo_slovar)

100%|██████████| 12766/12766 [16:11<00:00, 13.14it/s] 
